In [3]:
# Importing the required libraries
import pandas as pd
import numpy as np
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPool2D,AvgPool2D,BatchNormalization,Activation,Input
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.applications import InceptionV3

# 

# Heading to directory where data is stored 

In [18]:
cd ../input/task3

/kaggle/input/task3


# Model architecture 

In [19]:
model3=Sequential()
model3.add(Conv2D(32, (5, 5), padding="same",input_shape=(28,28,1)))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(Conv2D(32, (3, 3), padding="same"))
model3.add(BatchNormalization())
model3.add(MaxPool2D(pool_size=(2, 2)))
model3.add(Dropout(0.2))
model3.add(Conv2D(64, (3, 3), padding="same"))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(Conv2D(64, (3, 3), padding="same"))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(MaxPool2D(pool_size=(2, 2)))
model3.add(Dropout(0.2))
model3.add(Conv2D(128, (3, 3), padding="same"))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(Conv2D(128, (3, 3), padding="same"))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(MaxPool2D(pool_size=(2, 2)))
model3.add(Dropout(0.2))
model3.add(Conv2D(256, (3, 3), padding="same"))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(Conv2D(256, (3, 3), padding="same"))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(MaxPool2D(pool_size=(2, 2)))
model3.add(Dropout(0.2))
model3.add(Flatten())
model3.add(Dense(50))
model3.add(Activation("relu"))
model3.add(BatchNormalization())
model3.add(Dropout(0.3))
model3.add(Dense(10))
model3.add(Activation("softmax"))

# MNIST Handwritten Data

In [20]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()
import tensorflow as tf
test_X= test_X.reshape(10000,28,28,1)

# Applying onehotencoding on output data

In [21]:
from sklearn.preprocessing import OneHotEncoder
test_y=test_y.reshape(10000,1)
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
test_y = pd.DataFrame(enc.fit_transform(test_y).toarray())
# merge with main df bridge_df on key values

#bridge_df = bridge_df.join(enc_df)

# Data flow pipeline with addition of noise in input images (brightness and zoom). I have not included the rotation because it will completely change the data (i.e it wont belong to its assigned label)

In [22]:
datagen= ImageDataGenerator(
zoom_range=0.15,
brightness_range = [0.3,1.4],
validation_split=0.20)
train_it=datagen.flow_from_directory('mnistTask',class_mode='categorical',batch_size=100,target_size=(28,28),shuffle=True,color_mode='grayscale',subset='training')
test_it= datagen.flow(test_X,test_y,batch_size=100,shuffle=True)

Found 48006 images belonging to 10 classes.


# Saving the weights with max val accuracy

In [23]:
from keras.callbacks import ModelCheckpoint
filepath='../../working/IIIT.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list1 = [checkpoint]

# Variable learning rate with lr= 0.001, decay=0.001/200 and momentum =0.9

In [24]:
lr_schedule = tensorflow.keras.optimizers.SGD(
    lr=0.001,
    decay= 0.001/200,
    momentum=0.9)

# Compiling and training

In [25]:
model3.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)
model3.fit(train_it,steps_per_epoch=int(48006/100),epochs=50,validation_data=test_it,validation_steps=int(10000/100),callbacks=callbacks_list1)


Epoch 1/50
480/480 [==============================] - 94s 193ms/step - loss: 2.7724 - accuracy: 0.1014 - val_loss: 2.5012 - val_accuracy: 0.0189

Epoch 00001: val_accuracy improved from -inf to 0.01890, saving model to ../../working/IIIT.h5
Epoch 2/50
480/480 [==============================] - 60s 126ms/step - loss: 2.4372 - accuracy: 0.1059 - val_loss: 2.5238 - val_accuracy: 0.0078

Epoch 00002: val_accuracy did not improve from 0.01890
Epoch 3/50
480/480 [==============================] - 53s 111ms/step - loss: 2.3807 - accuracy: 0.1063 - val_loss: 2.5436 - val_accuracy: 0.0077

Epoch 00003: val_accuracy did not improve from 0.01890
Epoch 4/50
480/480 [==============================] - 54s 112ms/step - loss: 2.3479 - accuracy: 0.1070 - val_loss: 2.5690 - val_accuracy: 0.0028

Epoch 00004: val_accuracy did not improve from 0.01890
Epoch 5/50
480/480 [==============================] - 54s 112ms/step - loss: 2.3315 - accuracy: 0.1075 - val_loss: 2.6022 - val_accuracy: 7.0000e-04

Epoch 

# Max val accuracy reached to  1.8%

In [11]:
cd ../weight-task2

/kaggle/input/weight-task2


# Loading weights file

In [12]:
from keras.models import load_model
model_trained= load_model('mymodel2(7).h5')
model_trained.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)


In [13]:
cd ../task3

/kaggle/input/task3


# Save the model weights with max val accuracy

In [14]:
from keras.callbacks import ModelCheckpoint
filepath='../../working/weighttask.h5'
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

# Training the pretrained model (trained in part-1) and measuring val accuracy on mnist test set

In [15]:
model_trained.fit(train_it,steps_per_epoch=int(48006/100),epochs=50,validation_data=test_it,validation_steps=int(10000/100),callbacks=callbacks_list)

Epoch 1/50
480/480 [==============================] - 165s 336ms/step - loss: 3.0858 - accuracy: 0.1008 - val_loss: 2.7720 - val_accuracy: 0.0072

Epoch 00001: val_accuracy improved from -inf to 0.00720, saving model to ../../working/weighttask.h5
Epoch 2/50
480/480 [==============================] - 85s 177ms/step - loss: 2.4915 - accuracy: 0.1086 - val_loss: 2.8073 - val_accuracy: 0.0013

Epoch 00002: val_accuracy did not improve from 0.00720
Epoch 3/50
480/480 [==============================] - 83s 173ms/step - loss: 2.4153 - accuracy: 0.1094 - val_loss: 2.8374 - val_accuracy: 5.0000e-04

Epoch 00003: val_accuracy did not improve from 0.00720
Epoch 4/50
480/480 [==============================] - 83s 172ms/step - loss: 2.3629 - accuracy: 0.1113 - val_loss: 2.8734 - val_accuracy: 7.0000e-04

Epoch 00004: val_accuracy did not improve from 0.00720
Epoch 5/50
480/480 [==============================] - 84s 176ms/step - loss: 2.3409 - accuracy: 0.1125 - val_loss: 2.9440 - val_accuracy: 3.0

# The max validation accuracy of the model after training for 50 epochs with pretrained weights is 0.89% and with random initialization is 1.8%. The low accuracy is because each label directory consisted of all the numbers. For example instead of storing only 1 in label directory 1 it has images with all numbers from 0-9. Due to which the model is not able to distinguish each label directory properly which is leading to such poor validation accuracy. The image data is same as mnist data but is in shuffled manner. Each label is having all the numbers. Model is unable to distinguish between the labels.